Dans ce notebook, nous allons nous concentrer sur les variables

# Régression linéaire avec des variables catégorielles.

## Exercice 1

On suppose qu’on dispose d’un ensemble d’observations $(X_i, Y_i)$ avec $X_i, Y_i \in \mathbb{R}$. La régression linéaire consiste une relation linéaire $Y_i = a X_i + b + \epsilon_i$ qui minimise la variance du bruit. On pose :

$E(a, b) = \sum_i (Y_i - (a X_i + b))^2$

On cherche a, b tels que :

$a^*, b^* = \arg \min E(a, b) = \arg \min \sum_i (Y_i - (a X_i + b))^2$

La fonction est dérivable et on trouve :

$\frac{\partial E(a,b)}{\partial a} = - 2 \sum_i X_i ( Y_i - (a X_i + b)) \text{ et } \frac{\partial E(a,b)}{\partial b} = - 2 \sum_i ( Y_i - (a X_i + b))$

Il suffit alors d’annuler les dérivées. On résoud un système d’équations linéaires. On note :

$\begin{array}{l} \mathbb{E} X = \frac{1}{n}\sum_{i=1}^n X_i \text{ et } \mathbb{E} Y = \frac{1}{n}\sum_{i=1}^n Y_i \\ \mathbb{E}(X^2) = \frac{1}{n}\sum_{i=1}^n X_i^2 \text{ et } \mathbb{E}(XY) = \frac{1}{n}\sum_{i=1}^n X_i Y_i \end{array}$

Finalement :

$\begin{array}{l} a^* = \frac{ \mathbb{E}(XY) - \mathbb{E} X \mathbb{E} Y}{\mathbb{E}(X^2) - (\mathbb{E} X)^2} \text{ et } b^* = \mathbb{E} Y - a^* \mathbb{E} X \end{array}$


### 1. Nuage de points

In [22]:
import random
def generate_xy(n=100, a=0.5, b=1):
    res = []
    for i in range(0, n):
        x = random.uniform(0, 10)
        res.append((x, x*a + b + random.gauss(0,1)))
    return res

In [2]:
generate_xy(10)

[(9.71503802440293, 5.180449115456406),
 (0.8229493937313104, 2.104952592629796),
 (7.562197939489525, 4.858337668539051),
 (1.104593129497945, 1.1820863189746658),
 (5.063972466726126, 4.342027293327647),
 (2.953998094535769, 1.99343267227902),
 (2.0828805991320376, 2.5782863031931647),
 (7.126555110646074, 5.11537230430071),
 (2.3326949397696306, 2.4246543225536175),
 (4.876347364220336, 3.84236897777593)]

### 2. Calcul de l'esperance

Ecrire une fonction qui calcule $\mathbb{E} X, \mathbb{E} Y, \mathbb{E}(XY), \mathbb{E}(X^2)$.</br>
`obs` = résultat de la fonction précédente.

In [3]:
def calcule_exyxyx2(obs):
    sx = 0
    sy = 0
    sxy = 0
    sx2 = 0
    for x, y in obs:
        sx += x
        sy += y
        sxy += x * y
        sx2 += x * x
    n = len(obs)
    
    return sx/n, sy/n, sxy/n, sx2/n

In [4]:
obs = generate_xy(10)

calcule_exyxyx2(obs)

(3.8834813795387637, 3.292150703363384, 17.26833855662608, 24.07968652629133)

### 3. Calcul des coefs

Calculer les grandeurs $a^*$, $b^*$. A priori, on doit retrouver quelque chose d’assez proche des valeurs choisies pour la première question : $a=0.5$, $b=1$.

In [5]:
def calcule_ab(obs):
    sx, sy, sxy, sx2 = calcule_exyxyx2(obs)
    a = (sxy - sx * sx)  / (sx2 - sx**2)
    b = sy - a * sx
    return a, b

In [6]:
calcule_ab(obs)

(0.2430371203414089, 2.3483205719808007)

### 4. Generer les categories

In [8]:
import random

In [7]:
def generate_caty(n=100, a=0.5, b=1, cats=["rouge", "vert", "bleu"]):
    res = []
    for i in range(0, n):
        x = random.randint(0,2)
        cat = cats[x]
        res.append((cat, 10*x**2*a + b + random.gauss(0,1)))
        
    return res

In [9]:
generate_caty(10)

[('bleu', 20.811114549176946),
 ('rouge', 0.2601175645774494),
 ('rouge', 0.41017209362724594),
 ('vert', 6.6237061088962825),
 ('vert', 5.198308884804547),
 ('rouge', -0.3623775869062522),
 ('vert', 8.522424899291167),
 ('vert', 7.447112803327004),
 ('vert', 6.609531476419688),
 ('vert', 5.380269876915381)]

### 5. Regression

On voudrait quand même faire une régression de la variable $Y$ sur la variable catégorielle. On construit une fonction qui assigne un numéro quelconque mais distinct à chaque catégorie. La fonction retourne un dictionnaire avec les catégories comme clé et les numéros comme valeurs.

In [10]:
def numero_cat(obs):
    mapping = {}
    for color, y in obs:
        if color not in mapping:
            mapping[color] = len(mapping)
            
    return mapping

In [11]:
obs = generate_caty(100)

numero_cat(obs)

{'rouge': 0, 'bleu': 1, 'vert': 2}

### 6. Matrice $M_{ic}$

On construit la matrice $M_{ic}$ tel que : $M_{ic}$ vaut $1$ si c est le numéro de la catégorie $X_i$, $0$ sinon.

In [13]:
import numpy

def construit_M(obs):
    mapping = numero_cat(obs)
    M = numpy.zeros((len(obs), 3))
    for i, (color, y) in enumerate(obs):
        cat = mapping[color]
        M[i, cat] = 1.0
    return M

In [14]:
M = construit_M(obs)
M[:5]

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

### 7. Matrice Y

Il est conseillé de convertir la matrice $M$ et les $Y$ au format numpy. On ajoute un vecteur constant à la matrice $M$. La requête `numpy add column` sur un moteur de recherche vous directement à ce résultat : [**How to add an extra column to an numpy array**](https://stackoverflow.com/questions/8486294/how-do-i-add-an-extra-column-to-a-numpy-array).

In [15]:
def convert_numpy(obs):
    M = construit_M(obs)
    Mc = numpy.hstack([M, numpy.ones((M.shape[0], 1))])
    Y = numpy.array([y for c, y in obs])
    return M, Mc, Y.reshape((M.shape[0], 1))

In [16]:
M, Mc, Y = convert_numpy(obs)
Mc[:5], Y[:5]

(array([[1., 0., 0., 1.],
        [0., 1., 0., 1.],
        [1., 0., 0., 1.],
        [0., 0., 1., 1.],
        [1., 0., 0., 1.]]),
 array([[ 0.9650576 ],
        [20.19226629],
        [ 2.11220401],
        [ 5.77190133],
        [ 0.96008845]]))

### 8. Regression multidimensionnelle

On résoud la régression multidimensionnelle en appliquant la formule $C^* = (M'M)^{-1}M'Y$. La question 7 ne servait pas à grand chose excepté faire découvrir la fonction `hstack` car le rang de la matrice $Mc$ est `3` < `4`.

In [17]:
alpha = numpy.linalg.inv(M.T @ M) @ M.T @ Y
alpha

array([[ 1.09064326],
       [21.04949364],
       [ 6.03043917]])

### 9. Vecteur de la Regression

La régression détermine les coefficients $\alpha$ dans la régression $Y_i = \alpha_{rouge} \mathbf{1\!\!1}_{X_i = rouge} + \alpha_{vert} \mathbf{1\!\!1}_{X_i = vert} + \alpha_{bleu} \mathbf{1\!\!1}_{X_i = bleu} + \epsilon_i$.

Construire le vecteur $\hat{Y_i} = \alpha_{rouge} \mathbf{1\!\!1}_{X_i = rouge} + \alpha_{vert} \mathbf{1\!\!1}_{X_i = vert} + \alpha_{bleu} \mathbf{1\!\!1}_{X_i = bleu}$.

In [18]:
Yp = numpy.zeros((M.shape[0], 1))

for i in range(3):
    Yp[ M[:,i] == 1, 0] = alpha[i, 0]
    
Yp[:5]

array([[ 1.09064326],
       [21.04949364],
       [ 1.09064326],
       [ 6.03043917],
       [ 1.09064326]])

### 10. Vecteur des coefficients 

In [19]:
obs = [(x, y) for x, y in zip(Yp, Y)]
calcule_ab( obs )

(array([1.]), array([-8.8817842e-15]))

On aboutit au résultat $Y = \hat{Y} + \epsilon$. On a associé une valeur à chaque catégorie de telle sorte que la régression de $Y$ sur cette valeur soit cette valeur. Autrement dit, c’est la **meilleur approximation de Y sur chaque catégorie**. A quoi cela correspond-il ? C’est le second énoncé qui répond à cette question.

## Exercice 2

### 1. Nuage de points

In [21]:
import random

In [20]:
def generate_xy(n=100, a=0.5, b=1):
    res = []
    for i in range(0, n):
        x = random.uniform(0, 10)
        res.append((x, x*a + b + random.gauss(0,1)))
        
    return res

In [23]:
generate_xy(10)

[(5.11009010527384, 3.632014369964855),
 (7.0507183424317414, 4.308002633228412),
 (5.856953828673706, 2.279511231028116),
 (4.524854917882227, 1.986191110447013),
 (5.999355107240577, 2.894721034477197),
 (5.300995291610078, 3.615575559321312),
 (2.8985792983135203, 2.3299045312028035),
 (2.4970970481564714, 4.13689499633046),
 (6.1697010543766915, 4.004695536784815),
 (9.029745124136973, 4.978425360947978)]

### 2. Calcul de l'esperance

Ecrire une fonction qui calcule $\mathbb{E} X, \mathbb{E} Y, \mathbb{E}(XY), \mathbb{E}(X^2)$.</br>
`obs` = résultat de la fonction précédente.

In [24]:
def calcule_exyxyx2(obs):
    sx = 0
    sy = 0
    sxy = 0
    sx2 = 0
    for x, y in obs:
        sx += x
        sy += y
        sxy += x * y
        sx2 += x * x
    n = len(obs)
    
    return sx/n, sy/n, sxy/n, sx2/n

In [25]:
obs = generate_xy(10)

calcule_exyxyx2(obs)

(4.379486177890043, 3.6469888056971316, 17.45702767189654, 22.20018320698751)

### 3. Calcul des coefs

Calculer les grandeurs $a^*$, $b^*$. A priori, on doit retrouver quelque chose d’assez proche des valeurs choisies pour la première question : $a=0.5$, $b=1$.

In [26]:
def calcule_ab(obs):
    sx, sy, sxy, sx2 = calcule_exyxyx2(obs)
    a = (sxy - sx * sx)  / (sx2 - sx**2)
    b = sy - a * sx
    return a, b

In [27]:
calcule_ab(obs)

(-0.5704336070276463, 6.14519490307867)

### 4. Loi Multinomiale

On va simuler une loi multinomiale est de partir d’une **loi uniforme et discrète à valeur dans entre $1$ et $10$**. On tire un nombre, s’il est inférieur ou égal à $5$, ce sera la catégorie $0$, $1$ si c’est inférieur à $8$, $2$ sinon.

In [28]:
def generate_caty(n=100, a=0.5, b=1, cats=["rouge", "vert", "bleu"]):
    res = []
    for i in range(0, n):
        # on veut 50% de rouge, 30% de vert, 20% de bleu
        x = random.randint(1, 10)
        if x <= 5: x = 0
        elif x <= 8: x = 1
        else : x = 2
        cat = cats[x]
        res.append((cat, 10*x**2*a + b + random.gauss(0,1)))
    return res

In [29]:
obs = generate_caty(10)
obs

[('rouge', 2.337342914145855),
 ('rouge', 1.55148258645949),
 ('bleu', 19.049568247222407),
 ('vert', 4.41037809600245),
 ('rouge', 0.5575699793763214),
 ('rouge', 1.5802470680282303),
 ('vert', 7.016877520658216),
 ('vert', 4.8531401449009275),
 ('rouge', -0.10413089243197682),
 ('vert', 6.767519666681651)]

### 5. Histogramme

On voudrait quand même faire une régression de la variable $Y$ sur la **variable catégorielle**. On commence par les compter.</br> Construire une fonction qui compte le nombre de fois qu’une catégorie est présente dans les données (un **histogramme**).

In [31]:
def histogram_cat(obs):
    h = dict()
    for color, y in obs:
        h[color] = h.get(color, 0) + 1
    return h

In [32]:
histogram_cat(obs)

{'rouge': 5, 'bleu': 1, 'vert': 4}

### 6. Moyennes

Construire une fonction qui calcule la moyenne des $Y_i$ pour chaque catégorie : $\mathbb{E}(Y | rouge)$, $\mathbb{E}(Y | vert)$, $\mathbb{E}(Y | bleu)$.</br> La fonction retourne un **dictionnaire {couleur:moyenne}**.

In [33]:
def moyenne_cat(obs):
    h = dict()
    sy = dict()
    for color, y in obs:
        h[color] = h.get(color, 0) + 1
        sy[color] = sy.get(color, 0) + y
    for k, v in h.items():
        sy[k] /= v
    return sy

In [34]:
moyenne_cat(obs)

{'rouge': 1.184502331115584,
 'bleu': 19.049568247222407,
 'vert': 5.761978857060811}

### 7. Vecteur de Regression

Construire le vecteur $Z_i = \mathbb{E}(Y | rouge)\mathbf{1\!\!1}_{X_i = rouge} + \mathbb{E}(Y | vert) \mathbf{1\!\!1}_{X_i = vert} + \mathbb{E}(Y | bleu) \mathbf{1\!\!1}_{X_i = bleu}$.

In [35]:
moys = moyenne_cat(obs)
Z = [moys[c] for c, y in obs]
Z[:5]

[1.184502331115584,
 1.184502331115584,
 19.049568247222407,
 5.761978857060811,
 1.184502331115584]

### 8.Vecteur de coefficients

On utilise le résultat de la **question 3** pour calculer les coefficients de la régression $Y_i = a^* Z_i + b^*$.

In [36]:
obs2 = [(z, y) for (c, y), z in zip(obs, Z)]
calcule_ab( obs2 )

(1.0000000000000002, 0.0)

On aboutit au résultat $Y = \hat{Y} + \epsilon$. On a associé une valeur à chaque catégorie de telle sorte que la régression de $Y$ sur cette valeur soit cette valeur.

### 9. Matrice de Variance/Covariance

On calcule la **matrice de variance / covariance** pour les variables $(Y_i)$, $(Z_i)$, $(Y_i - Z_i)$, $\mathbf{1\!\!1}_{X_i = rouge}$, $\mathbf{1\!\!1}_{X_i = vert}$, $\mathbf{1\!\!1}_{X_i = bleu}$.

In [37]:
bigM = numpy.empty((len(obs), 6))
bigM[:, 0] = [o[1] for o in obs]
bigM[:, 1] = Z
bigM[:, 2] = bigM[:, 0] - bigM[:, 1]
bigM[:, 3] = [ 1 if o[0] == "rouge" else 0 for o in obs]
bigM[:, 4] = [ 1 if o[0] == "vert" else 0 for o in obs]
bigM[:, 5] = [ 1 if o[0] == "bleu" else 0 for o in obs]
bigM[:5]

array([[ 2.33734291,  1.18450233,  1.15284058,  1.        ,  0.        ,
         0.        ],
       [ 1.55148259,  1.18450233,  0.36698026,  1.        ,  0.        ,
         0.        ],
       [19.04956825, 19.04956825,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 4.4103781 ,  5.76197886, -1.35160076,  0.        ,  1.        ,
         0.        ],
       [ 0.55756998,  1.18450233, -0.62693235,  1.        ,  0.        ,
         0.        ]])

On utilise la fonction [np.cov](https://numpy.org/doc/stable/reference/generated/numpy.cov.html)

In [38]:
c = numpy.cov(bigM.T)
c

array([[ 3.12248367e+01,  3.02345428e+01,  9.90293886e-01,
        -2.00972067e+00,  4.26657477e-01,  1.58306319e+00],
       [ 3.02345428e+01,  3.02345428e+01,  2.35294310e-16,
        -2.00972067e+00,  4.26657477e-01,  1.58306319e+00],
       [ 9.90293886e-01,  2.35294310e-16,  9.90293886e-01,
        -7.40148683e-17,  1.15942957e-16, -1.72735535e-17],
       [-2.00972067e+00, -2.00972067e+00, -7.40148683e-17,
         2.77777778e-01, -2.22222222e-01, -5.55555556e-02],
       [ 4.26657477e-01,  4.26657477e-01,  1.15942957e-16,
        -2.22222222e-01,  2.66666667e-01, -4.44444444e-02],
       [ 1.58306319e+00,  1.58306319e+00, -1.72735535e-17,
        -5.55555556e-02, -4.44444444e-02,  1.00000000e-01]])

On affiche un peu mieux les résultats :

In [39]:
import pandas
pandas.DataFrame(c).applymap(lambda x: '%1.3f' % x)

,0,1,2,3,4,5
0,31.225,30.235,0.990,-2.010,0.427,1.583
1,30.235,30.235,0.000,-2.010,0.427,1.583
2,0.990,0.000,0.990,-0.000,0.000,-0.000
3,-2.010,-2.010,-0.000,0.278,-0.222,-0.056
4,0.427,0.427,0.000,-0.222,0.267,-0.044
5,1.583,1.583,-0.000,-0.056,-0.044,0.100


### 10. Construction des vecteurs de coefficients

On permute **rouge** et **vert**. Construire le vecteur $W_i = \mathbb{E}(Y | rouge)\mathbf{1\!\!1}_{X_i = vert} + \mathbb{E}(Y | vert)\mathbf{1\!\!1}_{X_i = rouge} + \mathbb{E}(Y | bleu)\mathbf{1\!\!1}_{X_i = bleu}$.

Utiliser le résultat de la **question 3** pour calculer les **coefficients de la régression** $Y_i = a^* W_i + b^*$. Vérifiez que l’erreur est supérieure.

In [40]:
moys = moyenne_cat(obs)
moys["rouge"], moys["vert"] = moys.get("vert", 0), moys.get("rouge", 0)

In [41]:
W = [moys[c] for c, y in obs]
obs3 = [(w, y) for (c, y), w in zip(obs, W)]
calcule_ab( obs3 )

(0.5726657616009273, 1.7899224051777747)

In [42]:
def calcule_erreur(obs):
    a, b = calcule_ab(obs)
    e = [(a*x + b - y)**2 for x, y in obs]
    return sum(e) / len(obs)

In [43]:
calcule_erreur(obs2), calcule_erreur(obs3)

(0.8912644970701278, 16.88869380547872)

C’est supérieur.

### Conclusion

L’[analyse des correspondances multiples](https://fr.wikipedia.org/wiki/Analyse_des_correspondances_multiples) est une façon d’étudier les modalités de variables catégorielles mais cela ne fait pas de la prédiction. Le modèle [logit - probit](https://fr.wikipedia.org/wiki/Mod%C3%A8le_probit) prédit une variable binaire à partir de variables continue mais dans notre cas, c’est la variable à prédire qui est continue. Pour effectuer une prédiction, il convertit les catégories en variables numériques (voir [Categorical Variables](https://en.wikipedia.org/wiki/Categorical_variable)). Le langage R est plus outillé pour cela : [Regression on categorical variables](https://www.r-bloggers.com/2013/01/regression-on-categorical-variables/). Le module [categorical-encoding](https://github.com/scikit-learn-contrib/category_encoders) est disponible en python. Cet examen décrit une méthode parmi d’autres pour transformer les catégories en variables continues.